# CHURN MODEL

## 1. Introduction 

In this homework, we consider the study case of churn problem. Namely, we want to predict whether a customer will churn to another company when his/her contract expires based on a data set consisting of some information features. Basically, we seperate our whole data into two parts: the traning data set and the test data set.

Firstly, we use our training data set to check the data information and then analyze the correlation between each feature by using Pandas. After dealing with the training data set, we set the features to be our $X$ and the LEAVE data to be our $Y$. Using these to fit our model. 

In particular, we will try two models: Logistic Regression and SVM. We want to compare them and see which one is better for our data so that we can get higher accuracy when we estimate our test data. 

## 2. Deal With the Data Set

In this section, we will import the training dataset, analyze the dataset and perpare the data in order to fit the model later. 

### 2.1 Import Training Data

We import pandas to read our csv data files and name our train data set as "df_train".

In [19]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import time

import numpy as np
from sklearn import preprocessing, decomposition,\
    linear_model, pipeline, metrics
    
DATA_DIR = "../data/"
df_train = pd.read_csv("data/train.csv")

In [20]:
df_train.info()
df_train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 12 columns):
COLLEGE                        18000 non-null object
INCOME                         18000 non-null int64
OVERAGE                        18000 non-null int64
LEFTOVER                       18000 non-null int64
HOUSE                          18000 non-null int64
HANDSET_PRICE                  18000 non-null int64
OVER_15MINS_CALLS_PER_MONTH    18000 non-null int64
AVERAGE_CALL_DURATION          18000 non-null int64
REPORTED_SATISFACTION          18000 non-null object
REPORTED_USAGE_LEVEL           18000 non-null object
CONSIDERING_CHANGE_OF_PLAN     18000 non-null object
LEAVE                          18000 non-null int64
dtypes: int64(8), object(4)
memory usage: 1.6+ MB


,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,LEAVE
0,zero,31953,0,6,313378,161,0,4,unsat,little,no,1
1,one,36147,0,13,800586,244,0,6,unsat,little,considering,1
2,one,27273,230,0,305049,201,16,15,unsat,very_little,perhaps,1
3,zero,120070,38,33,788235,780,3,2,unsat,very_high,considering,0
4,one,29215,208,85,224784,241,21,1,very_unsat,little,never_thought,1


By printing out the information and the headings of our train data set, we found that there are some objects instead of integers. Then we want to transfer the object into the catogory data as below. 

### 2.2 Catogorize Data by Using Catagory Numbers

After checking our data, we found that some of our data is represented as characters, we want to convert these into numbers. Thus, we replace the COLLEGE, REPORTED_SATISFACTION, REPORTED_USAGE_LEVEL and CONSIDERING_CHANGE_OF_PLAN colunmns into catagory numbers. Then print out the head of our data again to double check. 

In [21]:
# delete the negative fraud data from OVERAGE 
df_train = df_train[df_train.OVERAGE >= 0]

df_train['COLLEGE'] = df_train['COLLEGE'].replace({"one": 1, "zero": 0})
df_train['REPORTED_SATISFACTION'] = df_train['REPORTED_SATISFACTION'].replace({"unsat": -0.5, "very_unsat": -1,"very_sat":1,"avg":0,"sat":0.5})
df_train['REPORTED_USAGE_LEVEL'] = df_train['REPORTED_USAGE_LEVEL'].replace({"little": -0.5, "very_little": -1,"very_high":1,"avg":0,"high":0.5})
df_train['CONSIDERING_CHANGE_OF_PLAN'] = df_train['CONSIDERING_CHANGE_OF_PLAN'].replace({"no": -0.5, "considering": 0,"perhaps":0.5,"never_thought":-1,"actively_looking_into_it":1})
df_train.head()

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,LEAVE
0,0,31953,0,6,313378,161,0,4,-0.5,-0.5,-0.5,1
1,1,36147,0,13,800586,244,0,6,-0.5,-0.5,0.0,1
2,1,27273,230,0,305049,201,16,15,-0.5,-1.0,0.5,1
3,0,120070,38,33,788235,780,3,2,-0.5,1.0,0.0,0
4,1,29215,208,85,224784,241,21,1,-1.0,-0.5,-1.0,1


### 2.3 Analyze the Data

Then, we use the pandas convenient one-line method $corr()$ to calculate correlation between data frame columns. 

In [5]:
features = list(df_train.columns[:11])
for i in features[0:11]:
    corr= df_train[i].corr(df_train['LEAVE'])
    print(i,':',corr)

COLLEGE : -0.015381265908933713
INCOME : -0.09785130295420438
OVERAGE : -0.23201797602264684
LEFTOVER : -0.0602332121529971
HOUSE : 0.21080821161190402
HANDSET_PRICE : -0.09194575623331838
OVER_15MINS_CALLS_PER_MONTH : -0.1984867009311266
AVERAGE_CALL_DURATION : 0.006794284597085972
REPORTED_SATISFACTION : 0.01035083723080867
REPORTED_USAGE_LEVEL : 0.0021172996871939693
CONSIDERING_CHANGE_OF_PLAN : -0.0033917653657393435


From the correlation calculation above, we noticed that all coeffiencents do  not have very strong connection with LEAVE. However, we still can find that the OVERAGE and HOUSE have relative strong correlation with LEAVE. Also, REPORTED_USAGE_LEVEL and CONSIDERING_CHANGE_OF_PLAN have relatively weak correlation with LEAVE. 

However, since all coefficients do not have obvious correlation, we will not delete any features in our dataset. Instead, we will separate our dataset into three parts and then combine all of them into $X$ as shown below.

### 2.4 Use StandardScaler to Fit Data and Set as X

We first separate our data into three parts, the catagory data set, the Income and House set, others. Since the Income and House data numbers are kind of larger than others, we separate it into our second_part_train data. Then we use numpy hstack to stack arrays in sequence horizontally and named the whole thing to be our "x_train". Finally, we use standardScaler to scale our train data set. As shown below, we print out the data before and after applying standardscaler to compare. 

In [6]:
df_train.head()

dont_change_train = df_train[['COLLEGE','REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL',
                         'CONSIDERING_CHANGE_OF_PLAN']]

frist_part_train = df_train[['OVERAGE','LEFTOVER','HANDSET_PRICE','OVER_15MINS_CALLS_PER_MONTH',
                          'AVERAGE_CALL_DURATION']].as_matrix()

second_part_train = np.sqrt(df_train[['INCOME','HOUSE']]).as_matrix()

x_train = np.hstack((dont_change_train,frist_part_train,second_part_train))
standardScaler = preprocessing.StandardScaler()
print("before",x_train[:3,:])

standardScaler.fit(x_train)
x_train = standardScaler.transform(x_train)
print("after",x_train[:3,:])

before [[ 0.00000000e+00 -5.00000000e-01 -5.00000000e-01 -5.00000000e-01
   0.00000000e+00  6.00000000e+00  1.61000000e+02  0.00000000e+00
   4.00000000e+00  1.78754021e+02  5.59801751e+02]
 [ 1.00000000e+00 -5.00000000e-01 -5.00000000e-01  0.00000000e+00
   0.00000000e+00  1.30000000e+01  2.44000000e+02  0.00000000e+00
   6.00000000e+00  1.90123644e+02  8.94754715e+02]
 [ 1.00000000e+00 -5.00000000e-01 -1.00000000e+00  5.00000000e-01
   2.30000000e+02  0.00000000e+00  2.01000000e+02  1.60000000e+01
   1.50000000e+01  1.65145390e+02  5.52312412e+02]]
after [[-1.00685722 -0.34519328 -0.53951473 -0.89800055 -1.00052508 -0.66564605
  -1.07251763 -0.89581065 -0.45616125 -1.24509578 -0.64948249]
 [ 0.99318948 -0.34519328 -0.53951473 -0.11513162 -1.00052508 -0.40468461
  -0.68467971 -0.89581065 -0.00208132 -1.09511029  1.19518626]
 [ 0.99318948 -0.34519328 -1.20037806  0.66773731  1.6730904  -0.88932728
  -0.88560779  0.89792838  2.04127839 -1.42461775 -0.69072814]]


C:\Users\SuperBubble\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys
C:\Users\SuperBubble\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


### 2.5 Set the LEAVE column as Y

We print out the shape and first 10 elements of our LEAVE data to check. 

In [7]:
y_train = df_train['LEAVE'].as_matrix()
print(y_train.shape)
print(y_train[:10])

(17999,)
[1 1 1 0 1 1 1 1 1 0]


C:\Users\SuperBubble\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


## 3. Deal with Test Data

In this section, we do the same thing as above to prepare our test dataset.

###  Import, Catogory and Normalize the Test Data

By using similar idea, we import our test data, catogorize the object features and normalize our test data in order to fit our model to get the target prediction "LEAVE". 

In [8]:
## import data
df_test = pd.read_csv("test.csv")
df_test = df_test[df_test.OVERAGE >= 0]

## catogory data
df_test['COLLEGE'] = df_test['COLLEGE'].replace({"one": 1, "zero": 0})
df_test['REPORTED_SATISFACTION'] = df_test['REPORTED_SATISFACTION'].replace({"unsat": -0.5, "very_unsat": -1,"very_sat":1,"avg":0,"sat":0.5})
df_test['REPORTED_USAGE_LEVEL'] = df_test['REPORTED_USAGE_LEVEL'].replace({"little": -0.5, "very_little": -1,"very_high":1,"avg":0,"high":0.5})
df_test['CONSIDERING_CHANGE_OF_PLAN'] = df_test['CONSIDERING_CHANGE_OF_PLAN'].replace({"no": -0.5, "considering": 0,"perhaps":0.5,"never_thought":-1,"actively_looking_into_it":1})
df_test.head()

## normalize test data set
dont_change_test = df_test[['COLLEGE','REPORTED_SATISFACTION','REPORTED_USAGE_LEVEL',
                          'CONSIDERING_CHANGE_OF_PLAN']]

frist_part_test = df_test[['OVERAGE','LEFTOVER','HANDSET_PRICE','OVER_15MINS_CALLS_PER_MONTH',
                          'AVERAGE_CALL_DURATION']].as_matrix()
second_part_test = np.sqrt(df_test[['INCOME','HOUSE']]).as_matrix()

x_test = np.hstack((dont_change_test,frist_part_test,second_part_test))

print("before normalize",x_test[:3,:])
x_test = standardScaler.transform(x_test)
print("after normalize",x_test[:3,:])

before normalize [[ 1.00000000e+00 -1.00000000e+00 -1.00000000e+00  1.00000000e+00
   2.02000000e+02  0.00000000e+00  5.62000000e+02  2.90000000e+01
   1.30000000e+01  3.77896811e+02  5.14170205e+02]
 [ 0.00000000e+00 -5.00000000e-01  5.00000000e-01  1.00000000e+00
   6.60000000e+01  4.50000000e+01  1.90000000e+02  3.00000000e+00
   2.00000000e+00  2.03794014e+02  7.86884998e+02]
 [ 0.00000000e+00  0.00000000e+00 -1.00000000e+00 -5.00000000e-01
   1.91000000e+02  6.10000000e+01  3.55000000e+02  2.10000000e+01
   1.00000000e+00  2.27857412e+02  9.93150542e+02]]
after normalize [[ 0.99318948 -0.95835166 -1.20037806  1.45060623  1.34760678 -0.88932728
   0.80125355  2.35534134  1.58719845  1.38195078 -0.90078669]
 [-1.00685722 -0.34519328  0.78221191  1.45060623 -0.23331368  0.78828195
  -0.937008   -0.55948458 -0.91024118 -0.91477387  0.60112107]
 [-1.00685722  0.2679651  -1.20037806 -0.89800055  1.21973821  1.38476523
  -0.16600489  1.45847183 -1.13728115 -0.59733498  1.7370764 ]]


C:\Users\SuperBubble\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
C:\Users\SuperBubble\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


## 4. Fit Model Using Logistic Regression

After preparing the data, we first try to use the LogisticRegression. The score of the model is printed as below. 

In [9]:
#logistic
model = linear_model.LogisticRegressionCV(cv=10,verbose=5)
model.fit(x_train[:17000,:],y_train[:17000])
train_loss= model.score(x_train[:17000,:],y_train[:17000])
val_loss = model.score(x_train[:1000,:],y_train[:1000])
print(train_loss,val_loss)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s


0.6392941176470588 0.637


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.8s finished


As shown above, the predict score we get is 0.6392941176470588, which is not very good. Then we try to use a method called polynomial features for our x_train data to check if we can increase the accuracy. Basically, we generate a new feature matrix consisting of all polynomial combinations of the features with degree less than or equal to a specific degree.


In [10]:
polyFeaturizer = preprocessing.PolynomialFeatures(2)
polyFeaturizer.fit(x_train)
x_train_poly = polyFeaturizer.transform(x_train)
print(x_train_poly.shape)

x_test_poly = polyFeaturizer.transform(x_test)
print(x_test_poly.shape)

#logistic
scaler2 = preprocessing.StandardScaler()
x_train_poly_normalized = scaler2.fit_transform(x_train_poly)
x_test_poly_normalized = scaler2.transform(x_test_poly)
model = linear_model.LogisticRegressionCV(cv=10,verbose=5)
model.fit(x_train_poly_normalized[:17000,:],y_train[:17000])
print(model.score(x_train_poly_normalized[:17000,:],y_train[:17000]))
print(model.score(x_train_poly_normalized[:1000,:],y_train[:1000]))

(17999, 78)
(2000, 78)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.8s remaining:    0.0s


0.6694705882352942
0.652


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    2.1s finished


As shown above, the accurancy increases to 0.69 for a polynomial of degree less than 2. Then we tried a ploynomial with degree 3.

In [11]:
#logistic
polyFeaturizer = preprocessing.PolynomialFeatures(3)
polyFeaturizer.fit(x_train)
x_train_poly = polyFeaturizer.transform(x_train)
print(x_train_poly.shape)

x_test_poly = polyFeaturizer.transform(x_test)
print(x_test_poly.shape)

scaler2 = preprocessing.StandardScaler()
x_train_poly_normalized = scaler2.fit_transform(x_train_poly)
x_test_poly_normalized = scaler2.transform(x_test_poly)
model = linear_model.LogisticRegressionCV(cv=10,verbose=5)
model.fit(x_train_poly_normalized[:17000,:],y_train[:17000])
print(model.score(x_train_poly_normalized[:17000,:],y_train[:17000]))
print(model.score(x_train_poly_normalized[:1000,:],y_train[:1000]))

(17999, 364)
(2000, 364)


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    6.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    8.2s remaining:    0.0s


0.6888235294117647
0.673


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:   20.2s finished


By fitting model using Logistic Regression, we can conclude that the logistic regression does not work well for our data and higer degree of polynomials may cause the overfitting problem which is not general and good enough for the test data. 

In [12]:
predictions = model.predict(x_train_poly_normalized)
print(predictions.shape)

(17999,)


## 5. Fit Model Using SVM

After trying the Logistic Regression, we would like to try the SVM regression as below to check if this model can have a higher accuracy. 

In [13]:
#svm
from sklearn.svm import SVC
model = SVC()
model.fit(x_train[:17000,:],y_train[:17000])
train_loss= model.score(x_train[:17000,:],y_train[:17000])
val_loss = model.score(x_train[:1000,:],y_train[:1000])
print(train_loss,val_loss)

0.6984705882352941 0.698


We can see that the SVM model works better than the Logistic Regression model. The accurancy approximates to 0.69, which is not bad. 

At last, we save our prediction LEAVE data into the "prediation" array.

In [14]:
predictions = model.predict(x_test)
print(predictions.shape)

(2000,)


## 6. Write Prediction Into File

We define a "write_pred_to_file" function as below and write our predicitons above into the "test_submit.csv" file. 

In [15]:
def write_pred_to_file(y_pred,filename):
    fpt = open(filename,"w")
    fpt.write("ID,LEAVE\n")
    for i in range(y_pred.shape[0]):
        fpt.write(str(i)+","+str(int(y_pred[i]))+"\n")
write_pred_to_file(predictions,"test_submit.csv")

## 7. Conclusion

From above analysis, we can conclude that for our data set, the SVM has better prediction than the Logistic Regression. In addition, we cannot use polynomial with relatively high degree to increase our training data accurancy, since it will cause the overfitting problem and lose the general and accurancy for test data.

In the future, we will try to use confusion matrix to analyze the classification. Also, we would like to use the Tableau to virtualize the dataset. 